In [36]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [37]:
postgres_details = {
    "database": "benchbase",
    "user": "admin",
    "password": "password",
    "host": "localhost",  # "20.232.173.241",
    "port": 5432,
}

# Retrieving metrics

In [38]:
from dataclasses import fields
from autorocks.envs.postgres.metrics.bgwriter import PGStatBGWriter
import psycopg2
import psycopg2.extras

# connect to the db
from autorocks.envs.postgres.metrics.bgwriter import PGStatBGWriter
from autorocks.envs.postgres.metrics.database import PGStatDatabase
from autorocks.envs.postgres.metrics.dbconf import DatabaseConflicts
from autorocks.envs.postgres.metrics.statio_index import StatIOIndex
from autorocks.envs.postgres.metrics.statio_table import StatIOTable

connection = psycopg2.connect(**postgres_details)

cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

sql_context = f"""
SELECT * FROM pg_stat_database
"""
cursor.execute(sql_context)

# Fetch all rows from database
record = cursor.fetchall()
import pandas as pd

pd.DataFrame(record)

In [179]:
from dataclasses import fields
from autorocks.envs.postgres.metrics.bgwriter import PGStatBGWriter
import psycopg2
import psycopg2.extras

# connect to the db
from autorocks.envs.postgres.metrics.bgwriter import PGStatBGWriter
from autorocks.envs.postgres.metrics.database import PGStatDatabase
from autorocks.envs.postgres.metrics.dbconf import DatabaseConflicts
from autorocks.envs.postgres.metrics.statio_index import StatIOIndex
from autorocks.envs.postgres.metrics.statio_table import StatIOTable
from autorocks.envs.postgres.metrics.archiver import PGStatArchiver
from autorocks.envs.postgres.metrics.wal import PGStatWAL

METRICS_VIEWS = [
    # One per row
    PGStatArchiver,
    PGStatWAL,
    PGStatBGWriter,
    # requires filtering on database
    DatabaseConflicts,
    PGStatDatabase,
    # requires summing over
    StatIOIndex,
    StatIOTable,
]

database_name = "benchbase"
connection = psycopg2.connect(database=database_name, user="admin", password="password", host="localhost", port=5432)
cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
for metric in METRICS_VIEWS:
    # print(metric._view_name())
    sql_query = f"{metric.select_query()} {metric.from_query()} {metric.filter_query(database_name = database_name)}"
    cursor.execute(sql_query)
    # fetch one - the queries are designed to report one statistic anyway.
    record = cursor.fetchone()
    print(metric(**record))

In [ ]:
# print("Data from Database:- ", record)

In [26]:
import pandas as pd

pd.DataFrame(record)

## Setting Parameters

In [4]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

sql_context = """
select
    *
from
    pg_settings
"""


cursor.execute(sql_context)

record = cursor.fetchall()

pd.DataFrame(record)

In [32]:
pd.DataFrame(record).set_index("name").T["shared_buffers"]
## TODO: this can be useful since we have the min, max, and vartype

In [114]:
# Setting values
try:
    conn = psycopg2.connect(database="postgres", user="admin", password="password", host="localhost", port=5432)
    conn.set_session(autocommit=True)
    with conn.cursor() as cursor:
        for p in ["shared_buffers = DEFAULT", "wal_level = DEFAULT"]:
            cursor.execute(f"ALTER SYSTEM SET {p};")
    conn.commit()
finally:
    if conn:
        conn.close()
# issue restart command


# TODO: if developing locally use docker, if using a cloud host, ssh + issue ps_ctl
# https://manpages.ubuntu.com/manpages/trusty/man1/pg_ctl.1.html

import docker

docker_cli = docker.from_env()
cont = docker_cli.containers.get("postgres")
cont.restart()

In [39]:
from autorocks.envs.postgres.schema.ottertune_best import ottertune_best_result


query_search = " OR ".join([f"name =  '{k}'" for k in ottertune_best_result.keys()])

query_search

In [70]:
extra_tracking_params = {
    "track_counts": "on",
    "track_functions": "all",
    "track_io_timing": "on",
    "track_wal_io_timing": "on",
    "autovacuum": "off",
}

query_search_tracker = " OR ".join([f"name =  '{k}'" for k in extra_tracking_params.keys()])

In [40]:
try:
    with psycopg2.connect(**postgres_details) as conn:
        cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
        sql_context = f"""
        select
            name, setting, max_val, min_val
        from
            pg_settings
        where {query_search}
        """
        cursor.execute(sql_context)

        record = cursor.fetchall()
finally:
    if conn:
        conn.close()
pd.DataFrame(record)

In [21]:
ottertune_best_result

In [43]:
import autorocks.utils.converters as conv

conv.short_size_to_bytes("10gb")